In [26]:
#firstly, extracting all items inside a user's iventory (only cs2 items)
import requests
import time
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/117.0.0.0 Safari/537.36"
}
def fetch_inventory_page(steamid64: str, appid: int, contextid: int, start: str = None):
    url = f"https://steamcommunity.com/inventory/{steamid64}/{appid}/{contextid}"
    resp = requests.get(url,headers=HEADERS, timeout=15)
    resp.raise_for_status()
    return resp.json()
def inventory_items_for_user(steamid64: str, appid: int, contextid: int, delay_between_requests: float = 0.5):
    items = []
    seen_assetids = set()
    start = None

    while True:
        data = fetch_inventory_page(steamid64, appid, contextid, start)
        # check if inventory is private or empty
        if not data or "assets" not in data or "descriptions" not in data:
            # Might be private or zero items; return whatever we have or raise
            return items, data

        assets = data.get("assets", [])
        descriptions = data.get("descriptions", [])

        # Build lookup for descriptions by (classid, instanceid)
        desc_lookup = {}
        for d in descriptions:
            key = (str(d.get("classid")), str(d.get("instanceid", "0")))
            desc_lookup[key] = d

        for a in assets:
            assetid = a.get("assetid")  # unique per item
            if assetid in seen_assetids:
                continue
            seen_assetids.add(assetid)

            classid = str(a.get("classid"))
            instanceid = str(a.get("instanceid", "0"))
            desc = desc_lookup.get((classid, instanceid)) or {}

            item = {
                "assetid": assetid,
                "classid": classid,
                "instanceid": instanceid,
                "amount": a.get("amount", 1),
                "market_tradable_restriction": a.get("market_tradable_restriction"),
                # descriptive fields:
                "name": desc.get("name"),
                "type": desc.get("type"),
                "market_hash_name": desc.get("market_hash_name"),
                "tradable": desc.get("tradable"),
                "tags": desc.get("tags", []),
                "marketable": desc.get("marketable"),
                # entire desc for debugging
                "description_raw": desc
            }
            if item["tradable"] ==1 and item["marketable"] == 1:
                items.append(item)

        # pagination
        more = data.get("more_items") or False
        # some responses give 'last_assetid' to use as start; others use 'more_start'
        if more:
            start = data.get("last_assetid") or data.get("more_start")
            if not start:
                # fallback: if server indicates 'more_items' but no token, stop to avoid infinite loop
                break
            time.sleep(delay_between_requests)  # be polite
        else:
            break

    return items, data

In [27]:
item_summaries = []#this for latter fetch on marketoverpice
seen_names = set()#this for checking duplicates market_hash_name
for i, it in enumerate(items[:200], start=1):
    name = it['market_hash_name']
    #skip if name are already added
    if not name or name in seen_names:
        continue
    seen_names.add(name)
    summary = {
        "index": i,
        "name": it['name'],
        "market_hash_name": it['market_hash_name'],
        "tradable": it.get('tradable'),
        "marketable": it.get('marketable')
    }
    item_summaries.append(summary)

print(item_summaries)

[{'index': 1, 'name': 'Sticker | Aleksib (Gold) | Copenhagen 2024', 'market_hash_name': 'Sticker | Aleksib (Gold) | Copenhagen 2024', 'tradable': 1, 'marketable': 1}, {'index': 2, 'name': 'Revolution Case', 'market_hash_name': 'Revolution Case', 'tradable': 1, 'marketable': 1}, {'index': 4, 'name': 'MP5-SD | Dirt Drop', 'market_hash_name': 'MP5-SD | Dirt Drop (Field-Tested)', 'tradable': 1, 'marketable': 1}, {'index': 5, 'name': 'MP9 | Buff Blue', 'market_hash_name': 'MP9 | Buff Blue (Battle-Scarred)', 'tradable': 1, 'marketable': 1}, {'index': 6, 'name': 'Sticker | Nemiga (Foil) | Austin 2025', 'market_hash_name': 'Sticker | Nemiga (Foil) | Austin 2025', 'tradable': 1, 'marketable': 1}, {'index': 7, 'name': 'Sticker | broky (Foil) | Austin 2025', 'market_hash_name': 'Sticker | broky (Foil) | Austin 2025', 'tradable': 1, 'marketable': 1}, {'index': 8, 'name': 'Sticker | m0NESY (Foil) | Austin 2025', 'market_hash_name': 'Sticker | m0NESY (Foil) | Austin 2025', 'tradable': 1, 'marketable

In [28]:
#now, use the name of items, and fetch the current market price
#Note: we do need cookies for this request.
from urllib.parse import quote
def get_market_price(market_hash_name, appid=730, currency=1):
    base_url = "http://steamcommunity.com/market/priceoverview/"
    params = {
        "appid": appid,
        "currency": currency,
        "market_hash_name": market_hash_name
    }
    url = base_url + f"?appid={appid}&currency={currency}&market_hash_name=" + quote(market_hash_name)
    #We need the cookies input manually, or getting the cookies on a logged on browser onto steam automatically
    # cookie_str = ""

    # cookies = dict(item.split("=", 1) for item in cookie_str.split("; "))
    # resp = requests.get(url,headers=HEADERS,cookies=cookies)
    #update: we wont need cookies, just change the url to http, instead of https.
    resp = requests.get(url,headers=HEADERS)
    data = resp.json() 

    if data.get("success"):
        if not data.get("lowest_price") or not data.get("median_price"):
            return None
        else:
            return {
                "lowest_price": data.get("lowest_price"),
                "median_price": data.get("median_price"),
                "volume": data.get("volume")
            }

In [ ]:
import time
import random
from collections import defaultdict
data_by_timestamp = defaultdict(dict)
timestamp = int(time.time())
for item in item_summaries:
    name = item["market_hash_name"]
    price_info = get_market_price(name)
    
    if price_info:
        data_by_timestamp[timestamp][name] = price_info
    #     print(f"{name}: {price_info['lowest_price']} (median: {price_info['median_price']})")
    else:
        print(f"{name}: Price not found")
    #adding delays for bypass steam's rate limiting
    time.sleep(random.uniform(4.5, 6.0))

print("final data")
#this data now can be saved for further usage
#the data structure can be changed, since there can be alot more apis need to be fetch, but this is my intention for the final structure.
print(data_by_timestamp)

final data
defaultdict(<class 'dict'>, {1761501208: {'Sticker | Aleksib (Gold) | Copenhagen 2024': {'lowest_price': '$4.33', 'median_price': '$4.34', 'volume': '1'}, 'Revolution Case': {'lowest_price': '$0.60', 'median_price': '$0.57', 'volume': '117,044'}, 'MP5-SD | Dirt Drop (Field-Tested)': {'lowest_price': '$0.05', 'median_price': '$0.04', 'volume': '3,074'}, 'MP9 | Buff Blue (Battle-Scarred)': {'lowest_price': '$0.05', 'median_price': '$0.05', 'volume': '378'}, 'Sticker | Nemiga (Foil) | Austin 2025': {'lowest_price': '$0.24', 'median_price': '$0.23', 'volume': '402'}, 'Sticker | broky (Foil) | Austin 2025': {'lowest_price': '$0.21', 'median_price': '$0.28', 'volume': '68'}, 'Sticker | m0NESY (Foil) | Austin 2025': {'lowest_price': '$2.42', 'median_price': '$2.32', 'volume': '77'}, 'Sticker | Heavygod (Gold) | Austin 2025': {'lowest_price': '$2.57', 'median_price': '$2.60', 'volume': '1'}, 'Sticker | m0NESY | Austin 2025': {'lowest_price': '$0.21', 'median_price': '$0.24', 'volume